<a target="_blank" href="https://colab.research.google.com/github/TinArmEngineering/ltc-docs/blob/main/py-client/create-job.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# Starting up
Log into the website for Tin Arm Engineering's machine solver. Under **profile**, retrieve your API key and, optionally, the organisation id that you wish to use.  

**Consider your API key like a password. Do not commit it to a repository.**

In [1]:
pip install ltc_client pint-pandas

In [2]:
import ltc_client
from ltc_client.api import STATUS_JOB, JOB_STATUS
import numpy as np
import pint_pandas
import time
from google.colab import userdata



config = {  'root_url': "https://api.ltc.tinarmengineering.com",
            'api_key': userdata.get('api_key'),
            'org_id':userdata.get('org_id')}

# Initalise the API
api = ltc_client.Api(config["root_url"], config["api_key"], config["org_id"])

INFO:numexpr.utils:NumExpr defaulting to 2 threads.
INFO:root:root_url: https://api.ltc.tinarmengineering.com


## Unit Handling.
The API enforces correct units to avoid ambiguity and errors. This is made easy as the API is compatible with [Pint](https://pint.readthedocs.io/en/stable/) and  Pint-Pandas unit handling libraries.  

In [3]:
q = pint_pandas.PintType.ureg

## Set up the machine.
We will do a simulation of a concentrated wound fractional slot machine, we will get on to how to design the machine later.  The machine is comprised of:
1. the stator, and a
2. rotor,
3. and the winding parameters
### Stator
The tooth wound stator is parameterised by the following dictionary of parameters.

In [4]:
stator_parameters = {
    "slot_liner_thickness": 400 * q.um,
    "stator_bore": 8.20 * q.cm,
    "tooth_tip_depth": 1.0 * q.mm,
    "slot_opening": 1.5 * q.mm,
    "tooth_width": 9.8 * q.mm,
    "stator_outer_diameter": 0.136 * q.m,
    "back_iron_thickness": 5.5 * q.mm,
    "stator_internal_radius": 500 * q.um,
    "number_slots": 12 * q.count,
    "tooth_tip_angle": 70 * q.degrees
    }


### Rotor
A suitable rotor for this machine is a Surface Mounted rotor with Breadloaf Magnets.  It is parameterised by the following dictionary. We've used the variable `air_gap_length' to link the rotor_od to the stator bore.


In [5]:

air_gap_length = 1 * q.mm

rotor_parameters = {
    "rotor_od": stator_parameters["stator_bore"] - 2 * air_gap_length,
    "rotor_bore": 40 * q.mm,
    "banding_thickness": 0.5 * q.mm,
    "number_poles": 10 * q.count,
    "magnet_thickness": 4.5 * q.millimeter,
    "magnet_pole_arc": 150 * q.degrees,
    "magnet_inset": 0.25 * q.millimeter,
    "magnet_segments": 1 * q.count
    }

## Winding
The winding is described by the winding parametrs.  These need further explanation.  The parameter symmetry sets the degrees of symmetry used in the simulation.

In [6]:

# Calculate the GCD of the number of slots and the number of poles
symmetry = np.gcd(stator_parameters["number_slots"].magnitude, rotor_parameters["number_poles"].magnitude) * q.count

winding_parameters = {
    "symmetry": symmetry,
    "number_phases": 3 * q.count,
    "number_layers": 2 * q.count,
    "coil_span": 1 * q.count,
    "turns_per_coil": 43 * q.count,
    "empty_slots": 0 * q.count,
    "fill_factor": 42 * q.percent
    }

Finally create the Machine using the class `ltc_client.Machine`. This will initialise it with default materials.  We can change the materials later.

In [7]:

our_machine = ltc_client.Machine(stator_parameters, rotor_parameters, winding_parameters)

## Set up the simulation job
The simulation job is made up of a `Machine`, the opperating points, and the simulation parameters.

The Simulation Parameters control the length of the simulation and the number of transient timesteps. To calculate the ironloss we need to simulate over one full electrical period.   We also set the active length here as it is a 2d simulation.

In [8]:

simulation_parameters = {
    "samples_per_electrical_period": 45 * q.count/q.turn,
    "timestep_intervals": 45 * q.count,
    "active_length": 65 * q.mm,
    "magnet_segments": 2 * q.count}

## Operating Point
We will set off two simultanious simulation jobs, an open circut simulation with 0 current, and the nominal opperating point with a q axis current density of 6.23 $A/mm^{2}$.  We will have 2 simulation jobs

In [9]:
op_open_cct = {
    "simulated_speed": 2060 * q.rpm,
    "q_axis_current_density": 0 * q.A / q.mm ** 2,
    "d_axis_current_density": 0 * q.A / q.mm ** 2,
    "current_angle": 0 * q.degrees
    }

op_nominal = {
    "simulated_speed": 2060 * q.rpm,
    'q_axis_current_density': 6.23 * q.A * q.mm**-2,
    "d_axis_current_density": 0 * q.A / q.mm ** 2,
    'current_angle': 255 * q.degrees
    }

In [10]:

j1_open_cct = ltc_client.Job(our_machine, op_open_cct, simulation_parameters)
j2_nominal = ltc_client.Job(our_machine, op_nominal, simulation_parameters)

# Use the API to create and run the simulation job

The flow of a job state is :

* New: 0,
* QueuedForMeshing: 10,
* WaitingForMesh: 20,
* QueuedForSimSetup: 21,
* SimSetup: 22,
* QueuedForMeshConversion: 25,
* MeshConversion: 26,
* QueuedForSolving: 30,
* Solving: 40,
* QueuedForPostProcess: 50,
* PostProcess: 60,
* Complete: 70,

If a job encouters an error at any point, it is directed to a quarentined state
* Quarantined: 80


<div class="alert alert-block alert-info">
<b>Reusable artifacts</b> If the machine geometry is identical to one of your other machines, it will reuse the mesh, so that the results of different operating point simulations are comparable.  This is the case here so  the first job will create the mesh, the 2nd job will wait for the first job to be meshed, and then solve in parrallel.
</div>

In [11]:
j1_result = api.create_job(j1_open_cct)
j2_result = api.create_job(j2_nominal)

print('Job id, {id}, creation_date, {creation_date}, status {status}'.format(**j1_result))
print('Job id, {id}, creation_date, {creation_date}, status {status}'.format(**j2_result))

Job id, 67f665a34e0dceda698b0693, creation_date, 2025-04-09T12:18:43Z, status 0
Job id, 67f665a34e0dceda698b0694, creation_date, 2025-04-09T12:18:43Z, status 0


To start a job, we use the api call [`api.update_job_status(job_id, status)`](https://api.ltc.tinarmengineering.com/docs/index.html#/jobs/put_jobs__id__status__status_) and set the job status to `QueuedForMeshing` which is the value 10.  We use the dictionary `JOB_STATUS` as an enum.

Once the job is running, we probably want to monitor it.  Later we will show how to use the websockets to get asyncronous job updates without polling, but for this example, we will poll with 10s interval, and wait untill both jobs are in the status
`Completed` which has the value 70.

<div class="alert alert-block alert-info">
<b>Polling is not good</b> Please don't be tempted to use polling, it is not a good practice.  We will show you how to use websockets to get asyncronous updates in the next example.
</div>

In [12]:
j1_result = api.update_job_status(j1_open_cct.id, JOB_STATUS['QueuedForMeshing'])
j2_result = api.update_job_status(j2_nominal.id, JOB_STATUS['QueuedForMeshing'])

print('Job id= {id}, creation_date= {creation_date}, status= {status}'.format(**j1_result))
print('Job id= {id}, creation_date= {creation_date}, status= {status}'.format(**j2_result))

while STATUS_JOB[api.get_job(j1_open_cct.id)['status']] != 'Complete' or STATUS_JOB[api.get_job(j2_nominal.id)['status']] != 'Complete':
    print("job 1 {0} \t job 2 {1}".format(STATUS_JOB[api.get_job(j1_open_cct.id)['status']], STATUS_JOB[api.get_job(j2_nominal.id)['status']] ))
    time.sleep(10)
time.sleep(1)

INFO:root:Updating job status: https://api.ltc.tinarmengineering.com/jobs/67f665a34e0dceda698b0693/status/10?node_id=None&apikey=2ded6a97db3171e392ac45fce6d9d026&percentage_complete=None
INFO:root:Updating job status: https://api.ltc.tinarmengineering.com/jobs/67f665a34e0dceda698b0694/status/10?node_id=None&apikey=2ded6a97db3171e392ac45fce6d9d026&percentage_complete=None


Job id= 67f665a34e0dceda698b0693, creation_date= 2025-04-09T12:18:43Z, status= 21
Job id= 67f665a34e0dceda698b0694, creation_date= 2025-04-09T12:18:43Z, status= 21
job 1 MeshConversion 	 job 2 SimSetup
job 1 Solving 	 job 2 Solving
job 1 Solving 	 job 2 Solving
job 1 Solving 	 job 2 Solving
job 1 Solving 	 job 2 Solving
job 1 Solving 	 job 2 QueuedForPostProcess
job 1 Solving 	 job 2 PostProcess
job 1 PostProcess 	 job 2 Complete
